In [1]:
""" the aim of this project is to detect main characteristics for given picture and to return some pictures similar
 to reach our task, we use convolution neural network already trained by large picture corpus (ImageNet). 
 the highest layers learned to detect complex patern
 we flated the highest layer activation matrix and calculate the difference of array between the array associated to the 
 given picture and other pictures in our bank of images """

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import seaborn as sns
import os, sys
import pandas as pd 

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

from keras.layers import Flatten


from sklearn.utils import shuffle
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.models import Model

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as preprocess_input_Xception

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from PIL import Image, ImageDraw,  ImageFont

Using TensorFlow backend.


# Détection d'images de contenus similaires (brouillon)

On utilise des réseaux de neurones convolutifs pré-entrainés sur ImageNet. On utilise l'activation du dernier étage d'un réseau Xception dans le but d'obtenir un espace de similarité pertinant. 

In [2]:
# check the number of pictures in our bank of images

path = "./autre"
dirs = os.listdir( path )

nb_images = len(dirs)
    
print (nb_images)

3694


In [3]:
j_te = 0

# Open a file
path = "./autre"
dirs = os.listdir( path )

# Xception dim
dim = 299

X_pictures = np.zeros((len(dirs),dim,dim,3))

for file in (dirs):
           
    file_picture =  "./autre/" + str(file)
    #print (file_picture)
    
    img = Image.open(file_picture)
    
    img.thumbnail((dim, dim))
        
    #img = load_img(file_picture, target_size=(224, 224))  # this is a PIL image 
    matrix = img_to_array(img)
    X_pictures[j_te,0:matrix.shape[0],0:matrix.shape[1],:] = matrix[:,:,:]
        
    j_te = j_te + 1
    

In [4]:
imput_train_xception = preprocess_input_Xception(X_pictures)

In [5]:
XCEPTION = Xception(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
XCEPTION.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [7]:
intermediate_layer_model = Model(inputs=XCEPTION.input,
                                 outputs=XCEPTION.get_layer('avg_pool').output)

intermediate_layer_model.summary()
intermediate_output = intermediate_layer_model.predict(imput_train_resnet50[0:4,:,:,:])
print (intermediate_output.shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

block8_sepconv1_act (Activation (None, 19, 19, 728)  0           add_6[0][0]                      
__________________________________________________________________________________________________
block8_sepconv1 (SeparableConv2 (None, 19, 19, 728)  536536      block8_sepconv1_act[0][0]        
__________________________________________________________________________________________________
block8_sepconv1_bn (BatchNormal (None, 19, 19, 728)  2912        block8_sepconv1[0][0]            
__________________________________________________________________________________________________
block8_sepconv2_act (Activation (None, 19, 19, 728)  0           block8_sepconv1_bn[0][0]         
__________________________________________________________________________________________________
block8_sepconv2 (SeparableConv2 (None, 19, 19, 728)  536536      block8_sepconv2_act[0][0]        
__________________________________________________________________________________________________
block8_sep

(4, 2048)


In [ ]:
X_pictures = 0
matrix = np.zeros((len(dirs),2048))

# prédit deux par deux pour des raisons de mémoire et de puissance très limitées
for i in range (int(len(dirs)/2)):    
    if ((i%10) == 0 ):
        print (i)  

    matrix[2*i,:] =  intermediate_layer_model.predict(imput_train_resnet50[2*i:2*i+2,:,:,:])[0,:]
    matrix[2*i+1,:] =  intermediate_layer_model.predict(imput_train_resnet50[2*i:2*i+2,:,:,:])[1,:]   
    

In [9]:
pca = PCA(n_components=600)
pca.fit(matrix)

print (np.sum(pca.explained_variance_ratio_))
    

0.9546965520502182


In [10]:
X_pca = pca.transform(matrix)

In [11]:
np.save('matrix_autre.npy', X_pca)

In [2]:
mat = np.load('matrix_autre.npy')